In [19]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import numpy as np
import librosa
import librosa.display
import wave
import os
import matplotlib.pyplot as plt
import multiprocessing 

In [5]:
TRAIN_PATH= './data/urban-sound-classification/train/'
TEST_PATH= './data/urban-sound-classification/test/'

TRAIN_CSV = TRAIN_PATH+'train.csv'
TRAIN_FOLDER = TRAIN_PATH+'Train/'

TEST_CSV = TEST_PATH+'test.csv'
TEST_FOLDER = TEST_PATH+'Test/'

TARGETSampleRate = 22050

In [11]:
OUTPUT_FOLDER_CLASSES = './data/urban-sound-classification/newSpectrums224/'

In [31]:
Labels=[]
Training_File_Dict={}
with open(TRAIN_CSV) as f:
    readCSV = csv.reader(f, delimiter=',')
    for row  in readCSV:
        if(not row[0]=='ID'):
            Training_File_Dict[row[0]]=row[1]
            if(not row[1] in Labels):
                Labels.append(row[1])

In [32]:
PathDictrionary={}
for i in Labels:
    LabeldDirectory = OUTPUT_FOLDER_CLASSES+"train/"+i+"/"
    PathDictrionary[i]=LabeldDirectory
    if not os.path.exists(LabeldDirectory):
        os.makedirs(LabeldDirectory)

In [15]:
Training_File_Dict

{'0': 'siren',
 '1': 'street_music',
 '2': 'drilling',
 '3': 'siren',
 '4': 'dog_bark',
 '6': 'children_playing',
 '10': 'street_music',
 '11': 'drilling',
 '12': 'gun_shot',
 '15': 'dog_bark',
 '17': 'engine_idling',
 '18': 'siren',
 '19': 'gun_shot',
 '20': 'engine_idling',
 '22': 'air_conditioner',
 '24': 'air_conditioner',
 '26': 'engine_idling',
 '27': 'siren',
 '32': 'children_playing',
 '33': 'jackhammer',
 '35': 'gun_shot',
 '36': 'siren',
 '37': 'jackhammer',
 '38': 'street_music',
 '40': 'jackhammer',
 '42': 'engine_idling',
 '43': 'siren',
 '44': 'children_playing',
 '45': 'jackhammer',
 '46': 'engine_idling',
 '47': 'air_conditioner',
 '48': 'car_horn',
 '49': 'drilling',
 '50': 'siren',
 '52': 'gun_shot',
 '54': 'children_playing',
 '56': 'children_playing',
 '57': 'drilling',
 '59': 'gun_shot',
 '60': 'siren',
 '61': 'car_horn',
 '63': 'drilling',
 '65': 'street_music',
 '66': 'jackhammer',
 '67': 'children_playing',
 '68': 'dog_bark',
 '71': 'dog_bark',
 '72': 'street_mu

In [ ]:
### New Multithreaded
def log_mel_spec_tfm(fname, src_path, dst_path):

    x=[]
    sample_rate=0
    try:
        x ,sample_rate = librosa.load(src_path)
    except:
        print(fname+" Could not be computed!")
        return
    print(len(x),fname)
    n_fft = 1024
    hop_length = 256
    n_mels = 224
    fmin = 20
    fmax = TARGETSampleRate / 2 
    audio_data = librosa.resample(x, sample_rate, TARGETSampleRate)
    
    mel_spec_power = librosa.feature.melspectrogram(x, sr=sample_rate, n_fft=n_fft, 
                                                    hop_length=hop_length, 
                                                    n_mels=n_mels, power=2.0,
                                                   fmin=fmin,fmax=fmax)
    mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max) 
    width = np.shape(mel_spec_db)[1]
    if width >= n_mels:
        for i in range(0,int(np.floor(width/n_mels))):
            out= np.zeros(shape=(224,224))
            if((i+1)*n_mels<=width):
                out = mel_spec_db[:,(i)*n_mels:(i+1)*n_mels]
                im = plt.imshow(out)
                colors = im.cmap(im.norm(out))
                data = colors.astype(np.float64) / np.max(colors) # normalize the data to 0 - 1
                data = 255 * data # Now scale by 255
                img = data.astype(np.uint8)
                out = img[:,:,0:3] 
            plt.imsave(os.path.join(dst_path,(fname.replace(".wav",'-').replace(".m4a",'-').replace(".mp3",'-') +str(i)+ '.png')), out) 


counter = 0
filesToProcess=[]
for filename in os.listdir(TRAIN_FOLDER):
    if counter%100==0:
        print("Prepared thus far",counter)
    if filename.endswith(".wav") or filename.endswith(".m4a") or filename.endswith(".mp3"):
        filesToProcess.append(filename)
        counter +=1
    else:
        print("skipped: "+filename)
with multiprocessing.Pool(6) as p:
    outputDir = PathDictrionary[Training_File_Dict[filename.replace(".wav","")]]
    print(outputDir)
    p.starmap(log_mel_spec_tfm, [(f,TRAIN_FOLDER+'/'+str(f),PathDictrionary[Training_File_Dict[f.replace(".wav","")]]) for f in filesToProcess])
    print("Finished The Pool")
print("Done generating Spectra!")

Prepared thus far 0
Prepared thus far 100
Prepared thus far 200
Prepared thus far 300
Prepared thus far 400
Prepared thus far 500
Prepared thus far 600
Prepared thus far 700
Prepared thus far 800
skipped: .DS_Store
Prepared thus far 900
Prepared thus far 1000
Prepared thus far 1100
Prepared thus far 1200
Prepared thus far 1300
Prepared thus far 1400
Prepared thus far 1500
Prepared thus far 1600
Prepared thus far 1700
Prepared thus far 1800
skipped: 3121.png
Prepared thus far 1900
Prepared thus far 2000
Prepared thus far 2100
Prepared thus far 2200
Prepared thus far 2300
Prepared thus far 2400
Prepared thus far 2500
Prepared thus far 2600
Prepared thus far 2700
Prepared thus far 2800
Prepared thus far 2900
Prepared thus far 3000
Prepared thus far 3100
Prepared thus far 3200
Prepared thus far 3300
Prepared thus far 3400
Prepared thus far 3500
Prepared thus far 3600
Prepared thus far 3700
Prepared thus far 3800
Prepared thus far 3900
Prepared thus far 4000
Prepared thus far 4100
Prepared 

88200 3094.wav
32308 8676.wav
88200 5783.wav
88200 6461.wav
88200 5543.wav
88200 1134.wav
88200 506.wav
88200 317.wav
15215 1468.wav
88200 7802.wav
88200 8067.wav
88200 5352.wav
88200 7019.wav
88200 3685.wav
88200 471.wav
88200 7354.wav
88200 1652.wav
88200 4510.wav
88200 5592.wav
88200 6852.wav
88200 1446.wav
88200 1725.wav
88200 7223.wav
88200 4504.wav
88200 4854.wav
88200 459.wav
88200 6329.wav
88200 1043.wav
88200 8110.wav
88200 8458.wav
88200 260.wav
88200 3902.wav
88200 5146.wav
5275 7385.wav
53141 891.wav
88200 8701.wav
88200 1873.wav
88200 1938.wav
88200 3537.wav
88200 885.wav
88200 6475.wav
88200 4698.wav
88200 8462.wav
88200 1867.wav
88200 4299.wav
88200 7432.wav
12778 7751.wav
88200 2613.wav
46052 3654.wav
88200 5586.wav
88200 3916.wav
88200 4262.wav
88200 2830.wav
88200 7989.wav
88200 5225.wav
88200 4840.wav
88200 8670.wav
88200 7743.wav
88200 113.wav
88200 878.wav
88200 3640.wav
88200 3691.wav
88200 8470.wav
88200 7543.wav
80042 8476.wav
88200 1493.wav
88200 3898.wav
30209

88200 5988.wav
88200 3096.wav
88200 1686.wav
88200 4467.wav
2642 4921.wav
88200 271.wav
88200 7829.wav
88200 5750.wav
88200 2388.wav
88200 312.wav
88200 4301.wav
25073 472.wav
22712 4715.wav
88200 5583.wav
88200 714.wav
88200 6259.wav
88200 2770.wav
88200 6689.wav
88200 2364.wav
83543 4676.wav
88200 1294.wav
88200 6516.wav
88200 7597.wav
88200 2402.wav
88200 1536.wav
88200 6061.wav
31311 6851.wav
88200 1279.wav
88200 4073.wav
88200 3483.wav
88200 8274.wav
88200 1068.wav
88200 3929.wav
88200 2980.wav
88200 2945.wav
88200 8649.wav
88200 2017.wav
88200 7419.wav
88200 5380.wav
52180 5964.wav
88200 2389.wav
88200 3446.wav
88200 6664.wav
88200 5778.wav
88200 4110.wav
88200 728.wav
23373 4104.wav
88200 4920.wav
88200 5037.wav
88200 2416.wav
88200 6271.wav
88200 3246.wav
88200 3097.wav
88200 1251.wav
88200 4067.wav
88200 7609.wav
88200 2575.wav
88200 4529.wav
88200 1858.wav
88200 8513.wav
88200 3708.wav
88200 1278.wav
88200 8058.wav
88200 5637.wav
88200 1680.wav
70036 1679.wav
88200 7620.wav
8

88200 4164.wav
88200 821.wav
88200 4029.wav
88200 3232.wav
43169 2515.wav
16538 6402.wav
88200 5331.wav
88200 6946.wav
42738 7520.wav
38367 1425.wav
28353 61.wav
88200 8613.wav
88200 2273.wav
88200 7054.wav
88200 3998.wav
88200 565.wav
88200 2664.wav
88200 1151.wav
47812 1431.wav
88200 4602.wav
88200 4201.wav
88200 8615.wav
88200 7494.wav
24035 571.wav
88200 7040.wav
88200 8607.wav
88200 6015.wav
88200 835.wav
88200 6638.wav
88200 5283.wav
88200 6001.wav
88200 3226.wav
88200 1145.wav
88200 6416.wav
27909 3194.wav
88200 1157.wav
88200 5508.wav
8101 7917.wav
88200 4549.wav
88200 5323.wav
88200 4823.wav
88200 6370.wav
88200 4616.wav
88200 5682.wav
88200 3752.wav
88200 2138.wav
88200 217.wav
88200 2116.wav
88200 3179.wav
88200 3034.wav
88200 170.wav
88200 3540.wav
88200 1783.wav
88200 6358.wav
88200 4170.wav
88200 5445.wav
88200 7095.wav
32997 1973.wav
32855 1437.wav
88200 7726.wav
88200 2501.wav
74675 3595.wav
88200 1623.wav
88200 7046.wav
88200 559.wav
8068 1008.wav
88200 1357.wav
88200 

88200 5732.wav
88200 5733.wav
88200 4758.wav
88200 7067.wav
88200 560.wav
88200 3970.wav
88200 3343.wav
88200 5055.wav
88200 3037.wav
88200 8432.wav
88200 1792.wav
88200 8216.wav
64585 6820.wav
88200 8571.wav
15215 5497.wav
88200 2329.wav
88200 1370.wav
88200 3185.wav
88200 1554.wav
88200 4789.wav
88200 7251.wav
88200 7642.wav
88200 4832.wav
88200 2679.wav
88200 1232.wav
88200 7280.wav
24035 3567.wav
88200 8604.wav
88200 4363.wav
88200 3191.wav
88200 8570.wav
88200 7715.wav
88200 1829.wav
88200 6629.wav
88200 957.wav
46526 365.wav
88200 3425.wav
88200 4826.wav
88200 68.wav
88200 5320.wav
88200 6574.wav
88200 7294.wav
37485 196.wav
88200 5040.wav
88200 212.wav
88200 3751.wav
88200 6212.wav
88200 7073.wav
88200 371.wav
88200 6549.wav
88200 1786.wav
88200 3745.wav
88200 3215.wav
88200 4411.wav
88200 5483.wav
88200 1226.wav
88200 4198.wav
55913 6206.wav
88200 8610.wav
88200 5664.wav
88200 4945.wav
88200 6764.wav
88200 7657.wav
33884 7523.wav
88200 1569.wav
88200 5857.wav
88200 1183.wav
882

88200 985.wav
88200 4824.wav
88200 2308.wav
88200 8546.wav
88200 6179.wav
88200 6967.wav
88200 3390.wav
88200 1974.wav
22712 2930.wav
88200 948.wav
88200 1616.wav
88200 544.wav
88200 3805.wav
88200 7690.wav
88200 3770.wav
88200 7675.wav
88200 382.wav
88200 6973.wav
88200 6969.wav
24444 8154.wav
12789 236.wav
88200 761.wav
88200 8625.wav
34074 3811.wav
88200 3413.wav
28224 7516.wav
88200 7501.wav
88200 1952.wav
88200 1960.wav
88200 2924.wav
18462 3823.wav
88200 3407.wav
88200 6608.wav
88200 8632.wav
88200 1237.wav
88200 6783.wav
88200 1593.wav
88200 5076.wav
88200 5315.wav
88200 5316.wav
29327 3764.wav
88200 6558.wav
88200 1578.wav
12348 2479.wav
88200 784.wav
88200 5473.wav
88200 578.wav
88200 7646.wav
88200 6217.wav
59315 1210.wav
88200 1615.wav
88200 5302.wav
88200 6797.wav
88200 8575.wav
88200 7109.wav
88200 8552.wav
88200 381.wav
88200 8626.wav
88200 4180.wav
88200 3420.wav
88200 4400.wav
88200 7264.wav
88200 396.wav
88200 3189.wav
88200 8234.wav
88200 2910.wav
88200 2717.wav
88200

88200 5706.wav
88200 4806.wav
88200 787.wav
88200 5705.wav
88200 838.wav
88200 3363.wav
88200 232.wav
88200 6568.wav
88200 5063.wav
88200 3993.wav
17861 5712.wav
88200 7063.wav
40131 5707.wav
88200 554.wav
88200 7674.wav
88200 1574.wav
88200 2455.wav
88200 3376.wav
88200 186.wav
88200 6977.wav
88200 8547.wav
88200 8236.wav
88200 6633.wav
88200 7110.wav
88200 540.wav
88200 791.wav
88200 1412.wav
88200 7105.wav
88200 1148.wav
88200 1561.wav
24876 1205.wav
88200 7539.wav
88200 6963.wav
88200 3205.wav
88200 5074.wav
88200 6542.wav
88200 5934.wav
88200 4030.wav
88200 226.wav
88200 2084.wav
88200 8578.wav
88200 6224.wav
88200 2083.wav
88200 3987.wav
88200 2293.wav
88200 7893.wav
88200 4433.wav
88200 4394.wav
88200 4184.wav
88200 5933.wav
88200 2441.wav
50715 2042.wav
88200 6555.wav
88200 6583.wav
88200 3833.wav
52700 2327.wav
9607 5448.wav
88200 4427.wav
88200 7842.wav
88200 2733.wav
88200 778.wav
88200 4756.wav
88200 6793.wav
88200 4393.wav
88200 2056.wav
31275 6141.wav
23053 351.wav
88200 

88200 8068.wav
44460 807.wav
88200 5852.wav
88200 6974.wav
88200 3060.wav
88200 2487.wav
88200 4652.wav
88200 8409.wav
88200 4805.wav
88200 178.wav
33013 3548.wav
88200 8135.wav
88200 3941.wav
88200 5558.wav
88200 4227.wav
88200 219.wav
88200 2697.wav
88200 7206.wav
88200 3799.wav
88200 2237.wav
88200 4193.wav
88200 813.wav
88200 92.wav
54626 1677.wav
3087 7260.wav
88200 6839.wav
88200 4646.wav
88200 3830.wav
88200 6118.wav
88200 7048.wav
28004 5299.wav
88200 385.wav
39009 6051.wav
13451 2130.wav
88200 3969.wav
88200 485.wav
88200 6737.wav
88200 2318.wav
88200 4541.wav
15876 8653.wav
88200 6790.wav
88200 6811.wav
22491 7560.wav
88200 1058.wav
88200 5477.wav
88200 6948.wav
88200 1.wav
88200 4069.wav
88200 354.wav
88200 7429.wav
88200 1941.wav
88200 4769.wav
88200 5887.wav
88200 3706.wav
88200 1818.wav
88200 4650.wav
88200 3006.wav
88200 2290.wav
88200 8353.wav
88200 8182.wav
88200 4888.wav
88200 7060.wav
88200 3760.wav
88200 4839.wav
88200 2418.wav
88200 4960.wav
31973 4974.wav
88200 60

17547 5589.wav
88200 679.wav
88200 318.wav
30429 8443.wav
88200 8484.wav
88200 6292.wav
88200 6080.wav
18004 1937.wav
88200 7770.wav
88200 5835.wav
88200 330.wav
88200 652.wav
53802 2182.wav
88200 4867.wav
88200 695.wav
88200 4904.wav
88200 5943.wav
88200 134.wav
1323 1488.wav
88200 8322.wav
27685 4485.wav
31752 456.wav
88200 253.wav
88200 1301.wav
88200 5188.wav
88200 6875.wav
88200 1460.wav
88200 8085.wav
88200 2608.wav
88200 2792.wav
88200 5822.wav
88200 6902.wav
10584 4279.wav
88200 442.wav
88200 2169.wav
88200 2962.wav
88200 3516.wav
88200 2195.wav
88200 247.wav
88200 8694.wav
88200 3277.wav
88200 727.wav
88200 5173.wav
88200 5611.wav
88200 4873.wav
88200 3919.wav
88200 6469.wav
20713 7006.wav
24035 8457.wav
88200 8297.wav
88200 4640.wav
88200 8335.wav
88200 1306.wav
88200 859.wav
32533 1477.wav
88200 1868.wav
88200 8283.wav
43169 19.wav
88200 7349.wav
88200 6318.wav
88200 692.wav
88200 324.wav
88200 1473.wav
88200 733.wav
88200 1738.wav
10584 2976.wav
88200 685.wav
88200 8444.wav

79149 4046.wav
88200 3449.wav
88200 1701.wav
23721 295.wav
88200 3729.wav
88200 4326.wav
88200 6295.wav
88200 4679.wav
88200 5038.wav
88200 6889.wav
88200 7835.wav
88200 3102.wav
88200 6651.wav
88200 2795.wav
88200 8057.wav
88200 2757.wav
88200 6137.wav
14281 906.wav
88200 2351.wav
88200 7364.wav
88200 8508.wav
88200 720.wav
88200 7201.wav
88200 1715.wav
88200 6243.wav
88200 2971.wav
88200 4708.wav
88200 3664.wav
88200 734.wav
88200 907.wav
88200 5416.wav
88200 2025.wav
88200 6734.wav
88200 6524.wav
88200 4454.wav
88200 2379.wav
88200 4123.wav
88200 4333.wav
88200 5992.wav
88200 5370.wav
88200 2552.wav
32093 6256.wav
88200 8285.wav
88200 8654.wav
88200 3894.wav
88200 7148.wav
88200 2756.wav
88200 735.wav
88200 7567.wav
88200 4645.wav
26880 721.wav
88200 4327.wav
88200 6679.wav
88200 7358.wav
88200 6294.wav
88200 913.wav
88200 1116.wav
88200 1729.wav
88200 4483.wav
88200 6530.wav
88200 482.wav
88200 6046.wav
88200 2794.wav
88200 5011.wav
38367 709.wav
88200 8640.wav
88200 1289.wav
25552

In [ ]:
for i in range(0,2):
    print(i)